In [ ]:
# Calculate what the doppler drift would be for different frequencies 
# for transmitters traveling at different speeds and how that
# compares to the COSMIC 


In [ ]:
circumference = 6400 * 1000
s_in_day = 1 * 24 * 60 * 60